### Custom libs to be installed:

In [1]:
# !pip install rgf_python
# !pip install xgboost
# !pip install lightgbm

In [2]:
import os
import sys
import glob
import pathlib
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from rgf.sklearn import RGFClassifier
from sklearn.ensemble import RandomForestClassifier

/Users/elejke/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
sys.path.append(os.path.abspath('../src/'))
    
from data import prepare_submission
from data import data_generator_test
from data import data_generator_train

from model_selection import cross_val_score

from quality_estimator import BaseQualityEstimator

## Data preparation:

In [4]:
X, Xy, y = data_generator_train(root_dir="../data/ellipse/")

In [5]:
X_test, Xy_test = data_generator_test(root_dir="../data/ellipse/")

## Algorithm initialization:

In [6]:
# a number of 1-st stage meta-learners:
lgbm = LGBMClassifier(max_depth=2)
rgf = RGFClassifier(max_leaf=400, algorithm="RGF_Sib", test_interval=100, verbose=False)
rf = RandomForestClassifier(n_estimators=100)
xgb = XGBClassifier(max_depth=2, n_estimators=10)

# QualityEstimator definition with metrics family names to be used for feature search:
q_clf = BaseQualityEstimator(metrics=["dice_coefficient", "mae", "mse"], 
                             unary_metrics=[],
                             matching_metrics=[
                                 "match2tpr",
                                 "match2predBinRate",
                                 "match2gtBinRate"
                             ],
                             meta_clfs={'lgbm': lgbm, 
                                        'rgf': rgf,
                                        'rf': rf, 
                                        'xgb': xgb})

/Users/elejke/anaconda3/lib/python3.7/site-packages/rgf/utils.py:225: UserWarning: Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.
  warnings.warn("Cannot find FastRGF executable files. "


# Algorithm fitting 

* 1-st stage metrics: generation in the search space
* 2-nd stage metrics: aggregation for each generated metric in the search space
* feature selection from the aggregated metrics
* fitting of the 1-st level meta-estimators on the selected features
* aggregation function optimization over predictions of the 1-st level meta-estimators

In [7]:
q_clf.fit(X, Xy, y)

BaseQualityEstimator(matching_metrics=['match2tpr', 'match2predBinRate',
                                       'match2gtBinRate'],
                     meta_clfs={'lgbm': LGBMClassifier(max_depth=2),
                                'rf': RandomForestClassifier(),
                                'rgf': RGFClassifier(algorithm='RGF_Sib',
                                                     max_leaf=400,
                                                     verbose=False),
                                'xgb': XGBClassifier(base_score=0.5,
                                                     booster='gbtree',
                                                     colsample_bylevel=1,
                                                     colsample_bynode=1,
                                                     colsample_bytree=1,
                                                     gamma=...
                                                     interaction_constraints='',
                     

In [8]:
# predictions of the meta-classifier on test data:
y_pred = q_clf.predict(X_test, Xy_test)

In [9]:
# example of automatically selected features:
print(q_clf.selected_features['rf'])

['dice_coefficient' 'match2gtBinRate_0.01_coverageCalculation_iou_0.02'
 'match2gtBinRate_0.01_dice_score_iou_0.02'
 'match2gtBinRate_0.01_iou_iou_0.02'
 'match2gtBinRate_0.060000000000000005_coverageCalculation_iou_0.02'
 'match2gtBinRate_0.060000000000000005_dice_score_iou_0.02'
 'match2gtBinRate_0.060000000000000005_dice_score_iou_0.25'
 'match2gtBinRate_0.060000000000000005_iou_iou_0.02'
 'match2gtBinRate_0.060000000000000005_iou_iou_0.25'
 'match2gtBinRate_0.11_coverageCalculation_iou_0.02'
 'match2gtBinRate_0.11_coverageCalculation_iou_0.25'
 'match2gtBinRate_0.11_dice_score_iou_0.02'
 'match2gtBinRate_0.11_iou_iou_0.02' 'match2gtBinRate_0.11_iou_iou_0.25'
 'match2gtBinRate_0.16000000000000003_coverageCalculation_iou_0.02'
 'match2gtBinRate_0.16000000000000003_coverageCalculation_iou_0.25'
 'match2gtBinRate_0.16000000000000003_dice_score_iou_0.02'
 'match2gtBinRate_0.16000000000000003_iou_iou_0.02'
 'match2gtBinRate_0.21000000000000002_coverageCalculation_iou_0.02'
 'match2gtBinR

## Submission preparation:

In [10]:
root_dir = "../data/decomposed/"

SecretPart_427 = prepare_submission(y_pred, root_dir)

In [11]:
SecretPart_427.head(3)

,Case,Sample 1,Sample 2,Sample 3
0,00011827_003.png,1.00,1.00,1.00
1,00011925_072.png,4.25,4.75,3.25
2,00012045_019.png,1.00,5.00,1.00


In [ ]:
SecretPart_427.to_csv("../data/SecretPart_427.csv", index=False)